In [11]:
import numpy as np
import mxnet as mx
from mxnet import gluon
from mxnet.gluon.data import vision

def normalize(data, label):
    return data.astype(np.float32, copy=False)/(255.0/2) - 1.0, label.astype(np.float32)

preprocessing = vision.transforms.Compose([vision.transforms.Resize(size=(64, 64)),
                                           normalize])

batch_size = 64

mnist_train = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True, transform=preprocessing),
                                      batch_size, shuffle=True)
mnist_test = gluon.data.DataLoader(gluon.data.vision.MNIST(train=False, transform=preprocessing),
                                     batch_size, shuffle=False)

In [ ]:
# The downloaded data is of type `Dataset` which are
# Well suited to work with the new Gluon interface but less
# With the older symbol API, used in this tutorial.
# Therefore we convert them to numpy array first
# X = np.zeros((70000, 28, 28))
# for i, (data, label) in enumerate(mnist_train):
#     X[i] = data.asnumpy()[:,:,0]
# for i, (data, label) in enumerate(mnist_test):
#     X[len(mnist_train)+i] = data.asnumpy()[:,:,0]

In [ ]:
#Use a seed so that we get the same random permutation each time
# np.random.seed(1)
# p = np.random.permutation(X.shape[0])
# X = X[p]

In [ ]:
# import cv2
# X = np.asarray([cv2.resize(x, (64,64)) for x in X])

In [ ]:
# X = X.astype(np.float32, copy=False)/(255.0/2) - 1.0

In [ ]:
# X = X.reshape((70000, 1, 64, 64))
# X = np.tile(X, (1, 3, 1, 1))

In [ ]:
# import mxnet as mx
# batch_size = 64
# image_iter = mx.io.NDArrayIter(X, batch_size=batch_size)

In [14]:
class RandIter(mx.io.DataIter):
    def __init__(self, batch_size, ndim):
        self.batch_size = batch_size
        self.ndim = ndim
        self.provide_data = [('rand', (batch_size, ndim, 1, 1))]
        self.provide_label = []

    def iter_next(self):
        return True

    def getdata(self):
        #Returns random numbers from a gaussian (normal) distribution
        #with mean=0 and standard deviation = 1
        return [mx.random.normal(0, 1.0, shape=(self.batch_size, self.ndim, 1, 1))]
Z = 100
rand_iter = RandIter(batch_size, Z)

In [23]:
num_fc = 512
bias = True
bepsilon = 1e-5 + 1e-12

netG = gluon.nn.Sequential()
with netG.name_scope():
    netG.add(gluon.nn.Conv2DTranspose(1024, (4, 4), strides=(2,2), padding=(1,1), use_bias=bias))
    netG.add(gluon.nn.BatchNorm(epsilon=bepsilon))
    netG.add(gluon.nn.Activation('relu'))
    
    netG.add(gluon.nn.Conv2DTranspose(512, (4, 4), strides=(2,2), padding=(1,1), use_bias=bias))
    netG.add(gluon.nn.BatchNorm(epsilon=bepsilon))
    netG.add(gluon.nn.Activation('relu'))
    
    netG.add(gluon.nn.Conv2DTranspose(256, (4, 4), strides=(2,2), padding=(1,1), use_bias=bias))
    netG.add(gluon.nn.BatchNorm(epsilon=bepsilon))
    netG.add(gluon.nn.Activation('relu'))

    netG.add(gluon.nn.Conv2DTranspose(128, (4, 4), strides=(2,2), padding=(1,1), use_bias=bias))
    netG.add(gluon.nn.BatchNorm(epsilon=bepsilon))
    netG.add(gluon.nn.Activation('relu'))
    
    netG.add(gluon.nn.Conv2DTranspose(3, (4, 4), strides=(2,2), padding=(1,1), use_bias=bias))
    netG.add(gluon.nn.Activation('tanh'))

In [ ]:
# generatorSymbol = mx.sym.Activation(g5, name='gact5', act_type='tanh')

In [ ]:
data = mx.sym.Variable('data')

d1 = mx.sym.Convolution(data, name='d1', kernel=(4,4), stride=(2,2), pad=(1,1), num_filter=128, no_bias=no_bias)
dact1 = mx.sym.LeakyReLU(d1, name='dact1', act_type='leaky', slope=0.2)

d2 = mx.sym.Convolution(dact1, name='d2', kernel=(4,4), stride=(2,2), pad=(1,1), num_filter=256, no_bias=no_bias)
dbn2 = mx.sym.BatchNorm(d2, name='dbn2', fix_gamma=fix_gamma, eps=epsilon)
dact2 = mx.sym.LeakyReLU(dbn2, name='dact2', act_type='leaky', slope=0.2)

d3 = mx.sym.Convolution(dact2, name='d3', kernel=(4,4), stride=(2,2), pad=(1,1), num_filter=512, no_bias=no_bias)
dbn3 = mx.sym.BatchNorm(d3, name='dbn3', fix_gamma=fix_gamma, eps=epsilon)
dact3 = mx.sym.LeakyReLU(dbn3, name='dact3', act_type='leaky', slope=0.2)

d4 = mx.sym.Convolution(dact3, name='d4', kernel=(4,4), stride=(2,2), pad=(1,1), num_filter=1024, no_bias=no_bias)
dbn4 = mx.sym.BatchNorm(d4, name='dbn4', fix_gamma=fix_gamma, eps=epsilon)
dact4 = mx.sym.LeakyReLU(dbn4, name='dact4', act_type='leaky', slope=0.2)

d5 = mx.sym.Convolution(dact4, name='d5', kernel=(4,4), num_filter=1, no_bias=no_bias)
d5 = mx.sym.Flatten(d5)

label = mx.sym.Variable('label')
discriminatorSymbol = mx.sym.LogisticRegressionOutput(data=d5, label=label, name='dloss')

In [ ]:
#Hyper-parameters
sigma = 0.02
lr = 0.0002
beta1 = 0.5
# Define the compute context, use GPU if available
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()

#=============Generator Module=============
generator = mx.mod.Module(symbol=generatorSymbol, data_names=('rand',), label_names=None, context=ctx)
generator.bind(data_shapes=rand_iter.provide_data)
generator.init_params(initializer=mx.init.Normal(sigma))
generator.init_optimizer(
    optimizer='adam',
    optimizer_params={
        'learning_rate': lr,
        'beta1': beta1,
    })
mods = [generator]

# =============Discriminator Module=============
discriminator = mx.mod.Module(symbol=discriminatorSymbol, data_names=('data',), label_names=('label',), context=ctx)
discriminator.bind(data_shapes=image_iter.provide_data,
          label_shapes=[('label', (batch_size,))],
          inputs_need_grad=True)
discriminator.init_params(initializer=mx.init.Normal(sigma))
discriminator.init_optimizer(
    optimizer='adam',
    optimizer_params={
        'learning_rate': lr,
        'beta1': beta1,
    })
mods.append(discriminator)

In [ ]:
from matplotlib import pyplot as plt

#Takes the images in the batch and arranges them in an array so that they can be
#Plotted using matplotlib
def fill_buf(buf, num_images, img, shape):
    width = buf.shape[0]/shape[1]
    height = buf.shape[1]/shape[0]
    img_width = int(num_images%width)*shape[0]
    img_hight = int(num_images/height)*shape[1]
    buf[img_hight:img_hight+shape[1], img_width:img_width+shape[0], :] = img

#Plots two images side by side using matplotlib
def visualize(fake, real):
    #64x3x64x64 to 64x64x64x3
    fake = fake.transpose((0, 2, 3, 1))
    #Pixel values from 0-255
    fake = np.clip((fake+1.0)*(255.0/2.0), 0, 255).astype(np.uint8)
    #Repeat for real image
    real = real.transpose((0, 2, 3, 1))
    real = np.clip((real+1.0)*(255.0/2.0), 0, 255).astype(np.uint8)

    #Create buffer array that will hold all the images in the batch
    #Fill the buffer so to arrange all images in the batch onto the buffer array
    n = np.ceil(np.sqrt(fake.shape[0]))
    fbuff = np.zeros((int(n*fake.shape[1]), int(n*fake.shape[2]), int(fake.shape[3])), dtype=np.uint8)
    for i, img in enumerate(fake):
        fill_buf(fbuff, i, img, fake.shape[1:3])
    rbuff = np.zeros((int(n*real.shape[1]), int(n*real.shape[2]), int(real.shape[3])), dtype=np.uint8)
    for i, img in enumerate(real):
        fill_buf(rbuff, i, img, real.shape[1:3])

    #Create a matplotlib figure with two subplots: one for the real and the other for the fake
    #fill each plot with the buffer array, which creates the image
    fig = plt.figure()
    ax1 = fig.add_subplot(2,2,1)
    ax1.imshow(fbuff)
    ax2 = fig.add_subplot(2,2,2)
    ax2.imshow(rbuff)
    plt.show()

In [ ]:
# =============train===============
print('Training...')
for epoch in range(1):
    image_iter.reset()
    for i, batch in enumerate(image_iter):
        #Get a batch of random numbers to generate an image from the generator
        rbatch = rand_iter.next()
        #Forward pass on training batch
        generator.forward(rbatch, is_train=True)
        #Output of training batch is the 64x64x3 image
        outG = generator.get_outputs()

        #Pass the generated (fake) image through the discriminator, and save the gradient
        #Label (for logistic regression) is an array of 0's since this image is fake
        label = mx.nd.zeros((batch_size,), ctx=ctx)
        #Forward pass on the output of the discriminator network
        discriminator.forward(mx.io.DataBatch(outG, [label]), is_train=True)
        #Do the backward pass and save the gradient
        discriminator.backward()
        gradD = [[grad.copyto(grad.context) for grad in grads] for grads in discriminator._exec_group.grad_arrays]

        #Pass a batch of real images from MNIST through the discriminator
        #Set the label to be an array of 1's because these are the real images
        label[:] = 1
        batch.label = [label]
        #Forward pass on a batch of MNIST images
        discriminator.forward(batch, is_train=True)
        #Do the backward pass and add the saved gradient from the fake images to the gradient
        #generated by this backwards pass on the real images
        discriminator.backward()
        for gradsr, gradsf in zip(discriminator._exec_group.grad_arrays, gradD):
            for gradr, gradf in zip(gradsr, gradsf):
                gradr += gradf
        #Update gradient on the discriminator
        discriminator.update()

        #Now that we've updated the discriminator, let's update the generator
        #First do a forward pass and backwards pass on the newly updated discriminator
        #With the current batch
        discriminator.forward(mx.io.DataBatch(outG, [label]), is_train=True)
        discriminator.backward()
        #Get the input gradient from the backwards pass on the discriminator,
        #and use it to do the backwards pass on the generator
        diffD = discriminator.get_input_grads()
        generator.backward(diffD)
        #Update the gradients on the generator
        generator.update()

        #Increment to the next batch, printing every 50 batches
        i += 1
        if i % 5 == 0:
            print('epoch:', epoch, 'iter:', i)
            print
            print("   From generator:        From MNIST:")

            print("jes")
            # visualize(outG[0].asnumpy(), batch.data[0].asnumpy())
            print(outG[0].asnumpy())
            break